In [1]:
# Import modules with shorthand
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import xml.etree.ElementTree as ET
from functions import structure   # for functions you need 
# from functions import convert_top, convert_bottomDV, convert_bottomI, structure   # for functions you need 
# # to add (idx,data,tree,root,filename) when you call the function

from ClinicalProtocolDictionaries import TxSiteGroups, OrgansperTxSite, OrganColors, OrganCodes
from ClinicalProtocolDictionaries import TxSiteGroups, OrgansperTxSite, OrganColors, OrganCodes

#disable warnings
import warnings
warnings.filterwarnings("ignore")


#***********Select the file that contains the dose constraints you want to use to create the clinical protocol************

filepath='//kodiak/physics/Dose Constraints/Scripts/Structure Templates/'

csvfile='PACIFIC4'

structures = pd.read_csv(filepath + csvfile + '.csv')

finalfile = csvfile

TxArea = csvfile


In [2]:
#__________________________________ CREATE STRUCTURE TEMPLATE FOR CLINICAL PROTOCOL ___________________________________

#data info that will be used to create the XML structures:  
#
# 0 - structure name, 1 - type of structure, 2 - TypeIndex (always 2 for now), 3 - ColorAndStyle, 4 - DVHLineStyle (always 0 for now), 
# 5 - DVHLineColor (always -16777216), 6 - DVHLineWidth (always 1 for now)

#Create the XML matrix for the structure data
#Create the structure set based on the standardized list of organs suggested by PRO 2019 ASTRO consensus paper
# StructuresOfInterest=OrgansperTxSite[TxSite]

# or create the structure set based on the dose constraint sheet
StructuresOfInteresttemp=structures['Structure ID'].unique()


# add any additional structures beyond what the dose constraint sheet lists


if 'Thorax' in TxArea:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['PTV_doseincGy1',
          'PTV_doseincGy2',
          'PTV_doseincGy3']
         )
    print('Thorax')

elif 'Pelvis'  in TxArea:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['PTV_doseincGy1',
          'PTV_doseincGy2',
          'PTV_doseincGy3']
         )
    print('Pelvis' )

elif 'Brain' in TxArea:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
#               ['Glnd_Lacrimal',
#               'Hippocampus_L',
#               'Hippocampus_R',
#               'Pituitary',
#               'Scalp',
#               'A_Carotid',
          ['PTV_doseincGy1',
          'PTV_doseincGy2',
          'PTV_doseincGy3']
         )
    print('Brain') 

elif 'Abdomen'  in TxArea:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['PTV_doseincGy1',
          'PTV_doseincGy2',
          'PTV_doseincGy3']
         )
    print('Abdomen' )


elif 'Craniospinal'  in TxArea:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['PTV_doseincGy1',
          'PTV_doseincGy2',
          'PTV_doseincGy3']
         )
    print('Craniospinal' )


elif 'HN'  in TxArea:
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['PTV_doseincGy1',
          'PTV_doseincGy2',
          'PTV_doseincGy3']
         )
    print('HN' )

elif 'PACIFIC4'  in TxArea:  #this is the nomenclature specifically requested by the protocol
    StructuresOfInteresttemp=np.append(StructuresOfInteresttemp,
          ['PTV_DosexFractions',
          'IGTV_DosexFractions']
         )
    print('PACIFIC4' )

StructuresOfInterest=np.unique(StructuresOfInteresttemp)




PACIFIC4


In [6]:


# Create a dataframe to capture all the relevant information for the structures listed for that treatment site
XML_S= pd.DataFrame(0,index=range(0,len(StructuresOfInterest)),columns=
                    ['StrID','Type','TypeIdx','Color','DVHLS','DVHLC','DVHLW','iCode','iCodeScheme','iCodeSchemeVersion'])

# Set the structure names based on the list of organs for that treatment site
XML_S.StrID=StructuresOfInterest 

#  Set all the default values
XML_S.TypeIdx=2   
XML_S.DVHLS=0    #Style
XML_S.DVHLC= -16777216  #Color
XML_S.DVHLW=1    #Width
XML_S.iCodeScheme='FMA'
XML_S.iCodeSchemeVersion='3.2'

# find the colors that correspond to each organ using the OrganColors dictionary.  Organs that are derived like Lungs-ITV or 
# _PRV will have the same color as their original organs


XML_Stemp=XML_S 

        
av=0

for x in range (0,len(XML_Stemp),1):
    
    if XML_Stemp.StrID[x] in OrganColors: # if the structure is in the organ list, simply assign it the corresponding color
        XML_S.Color[x+av]=OrganColors[XML_Stemp.StrID[x]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[XML_Stemp.StrID[x]]
        
        if XML_Stemp.StrID[x].endswith('_L'):
            XML_S.DVHLS[x+av]=1
            
        
        
    elif '-' in XML_Stemp.StrID[x]: # if the structure is a derived structure from one found in the organ list,
                                #simply assign it the corresponding color
        temp=XML_Stemp.StrID[x].split('-') 
        
        XML_S.Color[x+av]=OrganColors[temp[0]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[temp[0]]
        XML_S.DVHLS[x+av]=2
        
        
        
        trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
        trueStr.StrID=temp[0]
        trueStr.Color=OrganColors[temp[0]]
        trueStr.Type='Organ'        
        trueStr.iCode=OrganCodes[temp[0]]
        
        XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
        av = av + 1

    elif '+' in XML_Stemp.StrID[x]: # if the structure is a derived structure from one found in the organ list,
                                #simply assign it the corresponding color
        temp=XML_Stemp.StrID[x].split('+') 
        
        XML_S.Color[x+av]=OrganColors[temp[0]]
        XML_S.Type[x+av]='Organ'
        XML_S.iCode[x+av]=OrganCodes[temp[0]]
        XML_S.DVHLS[x+av]=2
        
        
        
        trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
        trueStr.StrID=temp[0]
        trueStr.Color=OrganColors[temp[0]]
        trueStr.Type='Organ'        
        trueStr.iCode=OrganCodes[temp[0]]
        
        XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
        av = av + 1
        
        
    elif '_PRV' in XML_Stemp.StrID[x]: # if the structure is a derived structure split it first,                                 
        
        temp=XML_Stemp.StrID[x].split('_PRV') 
        print(temp)
        
#         if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color 
        if temp[0] in OrganColors: # if it's an organ, simply assign it the corresponding color             
            XML_S.Color[x+av]=OrganColors[temp[0]]
            XML_S.Type[x+av]='Organ'
            XML_S.iCode[x+av]='PRV'
            XML_S.DVHLS[x+av]=2
            
            trueStr=pd.DataFrame(XML_Stemp.loc[x,:]).transpose()
            
            trueStr.StrID=temp[0]
            trueStr.Color=OrganColors[temp[0]]
            trueStr.Type='Organ'        
            trueStr.iCode=OrganCodes[temp[0]] 
            trueStr.DVHLS[x+av]=0
        
            XML_S = pd.concat([XML_S.iloc[:x+av], trueStr, XML_S.iloc[(x+av):]], ignore_index=True)
            av = av + 1
            
    elif 'TV' in XML_Stemp.StrID[x]:
        XML_S.Type[x+av]='Target'
    
    else:
        print (StructuresOfInterest[x] + ' not found')
        
# Drop the duplicate structures      
if len(XML_S.StrID) != len(set(XML_S.StrID)):
    XML_S=XML_S.drop_duplicates(subset='StrID')  
    XML_S=XML_S.reset_index(drop=True) 

    
TargetColors=['Segment - Red','Segment - Orange','Segment - Pink', 'RGB255200150'] #set standard PTV colors

targets=XML_S[XML_S.Type.isin(['Target'])]  #find the structures set as "target" in the structure set from the dose constraints

ignorePorC=targets  #fill in a temporary array

# for x in range (0,len(targets),1):
#     ignorePorC.StrID[targets.index[x]]=targets.StrID[targets.index[x]][1:]  #take out the first letter of the target structure to give a given PTV, CTV, GTV
#                                               #the same color

for x in range (0,len(targets),1):
    Tind=targets.StrID[targets.index[x]].find('T')
    ignorePorC.StrID[targets.index[x]]=targets.StrID[targets.index[x]][Tind:]  #take out the letterS of the target structure before "TV" to create a given PTV, CTV, GTV
                                              #the same color
    
TargetVolumes=ignorePorC.StrID.unique()      #figure out how many unique targets we have

for x in range (0,len(TargetVolumes),1):
    XML_S.Color[XML_S['StrID'].str.contains(TargetVolumes[x])]=TargetColors[x]     #assigning them the specific colors
    
avt=0

#create GTV or CTV if there are none.

for x in range (0,len(TargetVolumes),1):
    
    avt=0

        
    if ('G'+TargetVolumes[x] not in XML_S['StrID']) and ('CW' not in TxArea) and ('Breast' not in TxArea) :    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)
        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()

        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='G' + temp[0]
        else:
            TVdef.StrID='G' + TargetVolumes[x]

        
        
        XML_S = pd.concat([XML_S.iloc[:idxTV+1+avt], TVdef, XML_S.iloc[(idxTV+1+avt):]], ignore_index=True)
        
        avt=1+avt

    if 'C'+TargetVolumes[x] not in XML_S['StrID']:    #create a GTV volume if they don't have one
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        if TargetVolumes[x].endswith('_Eval'):
            temp=TargetVolumes[x].split('_Eval')
            TVdef.StrID='C' + temp[0]
#             TVdef.StrID='P' + temp[0]
        else:
            TVdef.StrID='C' + TargetVolumes[x]            
        
        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        
        avt=1+avt
        
        
    if TargetVolumes[x].endswith('_Eval'):    #create a PTV volume without the _Eval
        
        idxTV=list(XML_S.StrID.isin(['P'+TargetVolumes[x]])).index(True)

        
        TVdef=pd.DataFrame(XML_S.loc[idxTV,:]).transpose()
        
        temp=TargetVolumes[x].split('_Eval')
        TVdef.StrID='P' + temp[0]           
        
        XML_S = pd.concat([XML_S.iloc[:idxTV+avt], TVdef, XML_S.iloc[(idxTV+avt):]], ignore_index=True)
        
        avt=1+avt
        
        

XML_S.Type[XML_S.StrID.str.startswith('PTV')]='PTV'
XML_S.iCode[XML_S.StrID.str.startswith('PTV')]='PTVp'


XML_S.Type[XML_S.StrID.str.startswith('ITV')]='CTV' #no ITV available at the moment
XML_S.iCode[XML_S.StrID.str.startswith('ITV')]='ITV'

XML_S.Type[XML_S.StrID.str.startswith('IGTV')]='GTV' #no IGTV available at the moment
XML_S.iCode[XML_S.StrID.str.startswith('IGTV')]='ITV'

XML_S.Type[XML_S.StrID.str.startswith('CTV')]='CTV'
XML_S.iCode[XML_S.StrID.str.startswith('CTV')]='CTVp'


XML_S.Type[XML_S.StrID.str.startswith('GTV')]='GTV'
XML_S.iCode[XML_S.StrID.str.startswith('GTV')]='GTVp'
XML_S.iCode[XML_S.StrID.str.startswith('GTVn')]='GTVn'
XML_S.DVHLS[XML_S.StrID.str.startswith('GTV')]=3


for x in range(0,len(XML_S),1):
    
    if not str.isdigit(str(XML_S.iCode [x])):
        XML_S.iCodeScheme[x]='99VMS_STRUCTCODE'
        XML_S.iCodeSchemeVersion[x]='1.0'




In [7]:
XML_S

,StrID,Type,TypeIdx,Color,DVHLS,DVHLC,DVHLW,iCode,iCodeScheme,iCodeSchemeVersion
0,Bowel,Organ,2,Contour - Cyan,0,-16777216,1,7199,FMA,3.2
1,BrachialPlex_L,Organ,2,RGB185122087,1,-16777216,1,45245,FMA,3.2
2,BrachialPlex_R,Organ,2,Brown,0,-16777216,1,45244,FMA,3.2
3,BrachialPlexus,Organ,2,RGB128 64 0,0,-16777216,1,5906,FMA,3.2
4,Bronchus,Organ,2,RGB 0204102,0,-16777216,1,7409,FMA,3.2
5,Bronchus+20,Organ,2,RGB 0204102,2,-16777216,1,7409,FMA,3.2
6,Esophagus,Organ,2,Contour - Orange,0,-16777216,1,7131,FMA,3.2
7,GreatVes,Organ,2,Blue,0,-16777216,1,GreatVessels,99VMS_STRUCTCODE,1.0
8,Heart,Organ,2,Contour - Mag,0,-16777216,1,7088,FMA,3.2
9,IGTV_DosexFractions,GTV,2,Segment - Red,0,-16777216,1,ITV,99VMS_STRUCTCODE,1.0


In [8]:
#use the XML file below as the template to create all clinical protocols:
StructureTemplate = ET.parse('Structure Template.xml')
tree=StructureTemplate
root=StructureTemplate.getroot()


# this just adds the correct extension to the file name for the clinical protocol
filename = finalfile +'.xml'
 

# Change the clinical protocol name to the final file name so it shows the correct name when it's imported into Eclipse:
root.find('.//Preview').set('ID', finalfile)
StructureTemplate.write(filename)

In [9]:
# This will create the structures in the XML file of the clinical protocol
idxs = 2

for x in range(0,len(XML_S),1):
    
    data=XML_S.loc[x,'StrID':'iCodeSchemeVersion'] #One structure per row of the dataframe
    
    if XML_S.Type[x] != 0: 
        
        structure(idxs,data,tree,root,filename)
        idxs= idxs + 1
        print('success!!! ' + str(x) + ':' + data[0] )  
        
    else: 
        print ('Skipped: ' + XML_S.StrID[x])  
         
        
        
# Save the structure template XML file in a specific folder

folderpath='//kodiak/physics/Dose Constraints/Scripts/Completed Structure Templates for Eclipse/'
StructureTemplate.write(folderpath + filename)

# # ___________________________ END OF THE STRUCTURE TEMPLATE CREATION IN XML FILE_______________________________________

   
    

success!!! 0:Bowel
success!!! 1:BrachialPlex_L
success!!! 2:BrachialPlex_R
success!!! 3:BrachialPlexus
success!!! 4:Bronchus
success!!! 5:Bronchus+20
success!!! 6:Esophagus
success!!! 7:GreatVes
success!!! 8:Heart
success!!! 9:IGTV_DosexFractions
success!!! 10:Liver
success!!! 11:Lung_L
success!!! 12:Lung_R
success!!! 13:Lungs
success!!! 14:Lungs-GTV
success!!! 15:PTV_DosexFractions
success!!! 16:CTV_DosexFractions
success!!! 17:GTV_DosexFractions
success!!! 18:Rib
success!!! 19:Skin
success!!! 20:SpinalCord
success!!! 21:Stomach
success!!! 22:Trachea


FileNotFoundError: [Errno 2] No such file or directory: '//kodiak/physics/Dose Constraints/Completed Structure Templates for Eclipse/PACIFIC4.xml'

In [ ]:
# # This creates the blank Structure template.  YOU DO NOT NEED TO RUN THIS UNLESS SOMETHING HAPPENS TO THE 
# # TEMPLATE AND IT NEEDS TO BE REDONE

# StructureTemplate = ET.parse('structure template test.xml')
# tree=StructureTemplate
# root=StructureTemplate.getroot()
# filename='Structure Template.xml'

# for child in Protocol.find('.//Structures'): 
#     Protocol.find('.//Structures').remove(child)
#     StructureTemplate.write(filename)

# # root.find('.//Structures').clear()
# # root.find('.//StructureTemplate').set('xmlns:xsi', 'http://www.w3.org/2001/XMLSchema-instance')
# # Protocol.find('.//Phases//Phase//FractionCount').clear()
# # root.find('.//Phases//Phase//FractionCount').text ='25'
# # Protocol.find('.//Phases//PlanTemplate//FractionCount').clear()
# # root.find('.//Phases//PlanTemplate//FractionCount').text ='25'

# root.find('.//Preview').set('ID', 'Structure Template')
# StructureTemplate.write(filename)
